## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kaitangata,NZ,-46.2817,169.8464,57.09,71,86,4.32,overcast clouds
1,Lazaro Cardenas,MX,17.9583,-102.2000,75.24,90,20,3.78,few clouds
2,Oktyabrskoye,RU,43.0645,44.7417,37.54,93,90,4.47,overcast clouds
3,Jamestown,US,42.0970,-79.2353,29.79,82,1,1.99,clear sky
4,Narsaq,GL,60.9167,-46.0500,28.62,95,100,11.56,snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input('What is the desired minimum temeprature? '))
max_temp = float(input('What is the desired maximum temeprature? '))

What is the desired minimum temeprature? 60
What is the desired maximum temeprature? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

desired_temps_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
desired_temps_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Lazaro Cardenas,MX,17.9583,-102.2000,75.24,90,20,3.78,few clouds
10,San Ignacio,BZ,17.1561,-89.0714,66.34,94,0,0.00,clear sky
15,Mangrol,IN,21.1167,70.1167,85.42,43,4,6.91,clear sky
16,Camacha,PT,33.0833,-16.3333,63.54,68,75,14.97,broken clouds
22,Rikitea,PF,-23.1203,-134.9692,76.33,84,91,16.40,overcast clouds
...,...,...,...,...,...,...,...,...,...
1918,San Jose De Buan,PH,12.0529,125.0251,78.75,89,100,5.17,moderate rain
1922,Alamos,MX,27.0167,-108.9333,61.45,91,11,2.39,few clouds
1923,Nouakchott,MR,18.0858,-15.9785,73.33,42,100,11.34,overcast clouds
1924,Eydhafushi,MV,5.1033,73.0708,81.70,70,100,16.55,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
desired_temps_df.count()

City                   988
Country                981
Lat                    988
Lng                    988
Max Temp               988
Humidity               988
Cloudiness             988
Wind Speed             988
Current Description    988
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_desired_temps_df = desired_temps_df.dropna()
clean_desired_temps_df.count()

City                   981
Country                981
Lat                    981
Lng                    981
Max Temp               981
Humidity               981
Cloudiness             981
Wind Speed             981
Current Description    981
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_desired_temps_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lazaro Cardenas,MX,75.24,few clouds,17.9583,-102.2000,
10,San Ignacio,BZ,66.34,clear sky,17.1561,-89.0714,
15,Mangrol,IN,85.42,clear sky,21.1167,70.1167,
16,Camacha,PT,63.54,broken clouds,33.0833,-16.3333,
22,Rikitea,PF,76.33,overcast clouds,-23.1203,-134.9692,
24,Victoria,HK,76.17,clear sky,22.2855,114.1577,
26,Carnarvon,AU,89.67,clear sky,-24.8667,113.6333,
28,Esperance,AU,67.84,clear sky,-33.8667,121.9000,
31,Chicama,PE,67.06,overcast clouds,-7.8447,-79.1469,
32,Ixtapa,MX,69.76,few clouds,20.7000,-105.2000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    #add the latitude and longitude to location key for the params dicitonary
    params['location'] = f'{lat},{long}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except IndexError:
        print('Hotel not found... Skipping')
       
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... S

In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lazaro Cardenas,MX,75.24,few clouds,17.9583,-102.2000,quinta antigua
10,San Ignacio,BZ,66.34,clear sky,17.1561,-89.0714,Rainforest Haven Inn
15,Mangrol,IN,85.42,clear sky,21.1167,70.1167,Hotel The Royal Honours
16,Camacha,PT,63.54,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
22,Rikitea,PF,76.33,overcast clouds,-23.1203,-134.9692,People ThankYou


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))